In [1]:
영등포구 = [1156010100,1156010200,1156010300,1156010400,1156010500,1156010600,1156010700,1156010800,1156010900,1156011000,1156011100,1156011200,1156011300,1156011400,1156011500,1156011600,1156011700,1156011800,1156011900,1156012000,1156012100,1156012200,1156012300,1156012400,1156012500,1156012600,1156012700,1156012800,1156012900,1156013000,1156013100,1156013200,1156013300,1156013400]
yeongdeungpo = ['영등포동','영등포동1가','영등포동2가','영등포동3가','영등포동4가','영등포동5가','영등포동6가','영등포동7가','영등포동8가','여의도동','당산동1가','당산동2가','당산동3가','당산동4가','당산동5가','당산동6가','당산동','도림동','문래동1가','문래동2가','문래동3가','문래동4가','문래동5가','문래동6가','양평동1가','양평동2가','양평동3가','양평동4가','양평동5가','양평동6가','양화동','신길동','대림동','양평동']

In [2]:
import requests #필요한 모듈들을 import 합니다.
from bs4 import BeautifulSoup
import pandas as pd

names = []
areas = []
prices = []
area_per_price = []
for area_code in 영등포구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
    
    names.append(new_name) # lists in lists 로 저장합니다
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(yeongdeungpo)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)

kk = {'dong':영등포구,'per price':amean}
df = pd.DataFrame(kk,index = yeongdeungpo)
df

,dong,per price
영등포동,1156010100,16773642
영등포동1가,1156010200,17396444
영등포동2가,1156010300,12715978
영등포동3가,1156010400,11993359
영등포동4가,1156010500,22241176
영등포동5가,1156010600,17285714
영등포동6가,1156010700,6661491
영등포동7가,1156010800,18214122
영등포동8가,1156010900,256703114
여의도동,1156011000,25555863


In [3]:
df.to_csv('yeongdeungpogu.csv')